In [1]:
import pandas as pd
import numpy as np

In [2]:
###################### Data Preprocessing ############################

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
###################### Model Development ###############################

from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [4]:
################## Accuracy Check #######################
from sklearn.metrics import r2_score

In [5]:
################ Warnings Ingore ############

import warnings
warnings.filterwarnings('ignore')

# Data Import 

In [6]:
data = pd.read_csv('SecondCar.csv')
data.head(2)

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.0,106001,100000
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.0,108556,100000


# Data Preprocessing

    1. Data duplicates -- remove
    2. Missing value > 75% -->> remove column
    3. Missing value treatment
    4. Unique Value based removal -->> when unique value == 1 or unique value == len(data)
    5. Feature engineering -->> variety reduction
    6. Label Encoding -->> Alphabetical order converted to numbers
    6.5. Correlation -->> 
    

# Step 1. Duplicate Data

In [7]:
print('Before duplicate removal -->>', len(data))

Before duplicate removal -->> 4340


In [8]:
data = data.drop_duplicates(keep='first')

In [9]:
print('After duplicate removal -->>', len(data))

After duplicate removal -->> 4340


# Step 2. Missing value > 75%

In [10]:
data.isna().sum()

name                0
year                0
km_driven           0
fuel                3
seller_type         2
transmission        2
owner               3
Rating              4
ExShowroom Price    0
selling_price       0
dtype: int64

In [11]:
missing_value_df = pd.DataFrame(data.isna().sum(), columns=['missing_value_count'])
missing_value_df

,missing_value_count
name,0
year,0
km_driven,0
fuel,3
seller_type,2
transmission,2
owner,3
Rating,4
ExShowroom Price,0
selling_price,0


In [12]:
missing_value_df['perc'] = 100*missing_value_df['missing_value_count']/len(data)
missing_value_df

,missing_value_count,perc
name,0,0.000000
year,0,0.000000
km_driven,0,0.000000
fuel,3,0.069124
seller_type,2,0.046083
transmission,2,0.046083
owner,3,0.069124
Rating,4,0.092166
ExShowroom Price,0,0.000000
selling_price,0,0.000000


##### filter dataframe based on threshold value

In [13]:
missing_value_df[missing_value_df['perc']>75]

,missing_value_count,perc


##### Store list of columns to be deleted

In [14]:
columns_to_be_deleted = list(missing_value_df[missing_value_df['perc']>75].index)
columns_to_be_deleted

[]

##### Delete columns from original data

In [15]:
data.head(2)

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.0,106001,100000
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.0,108556,100000


In [16]:
data.drop(columns=columns_to_be_deleted, inplace=True)
data.head(2)

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.0,106001,100000
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.0,108556,100000


In [17]:
# missing_value_df = pd.DataFrame(data.isna().sum(), columns=['missing_value_count'])
# missing_value_df['perc'] = 100*missing_value_df['missing_value_count']/len(data)
# columns_to_be_deleted = list(missing_value_df[missing_value_df['perc']>75].index)
# data.drop(columns=columns_to_be_deleted, inplace=True)

# Step 3. Missing Value Treatment

In [18]:
data.isna().sum()

name                0
year                0
km_driven           0
fuel                3
seller_type         2
transmission        2
owner               3
Rating              4
ExShowroom Price    0
selling_price       0
dtype: int64

In [19]:
for col in data.columns:
    
    if data[col].dtype == 'object':
#         print(col, '-->>', data[col].dtype, '-->>', data[col].mode()[0])
        data[col].fillna(data[col].mode()[0], inplace = True)
    
    if data[col].dtype != 'object':
#         print('*'*40, col, '-->>', data[col].dtype, '-->>', data[col].median())
        data[col].fillna(data[col].median(), inplace = True)

In [20]:
data.isna().sum()

name                0
year                0
km_driven           0
fuel                0
seller_type         0
transmission        0
owner               0
Rating              0
ExShowroom Price    0
selling_price       0
dtype: int64

# Mean, Mode, Median

In [21]:
data['fuel'].mode()[0]

'Diesel'

In [22]:
data['km_driven'].median()

60000.0

In [23]:
data['km_driven'].mean()

66215.77741935484

# Unique Value Check

In [24]:
data['dummy_col'] = 'vinay'
data.head(2)

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,dummy_col
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.0,106001,100000,vinay
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.0,108556,100000,vinay


In [25]:
data['dummy_col'].unique()

array(['vinay'], dtype=object)

In [26]:
for col in data.columns:
    if (data[col].nunique() == 1):
        del data[col]
    elif ((data[col].nunique() == len(data)) & (data[col].dtype == 'object')):
        del data[col]

# Feature Engineering

In [27]:
for col in data.columns:
    print(col, '-->>', data[col].nunique(), '-->>', data[col].dtype)

name -->> 1491 -->> object
year -->> 27 -->> int64
km_driven -->> 770 -->> int64
fuel -->> 5 -->> object
seller_type -->> 3 -->> object
transmission -->> 2 -->> object
owner -->> 5 -->> object
Rating -->> 566 -->> float64
ExShowroom Price -->> 4331 -->> int64
selling_price -->> 445 -->> int64


# Object Column Variety reduction --->> Needs to be done manually

In [28]:
data['name']

0                          Tata Indica Vista Aqua 1.4 TDI
1                                       Tata Nano Lx BSIV
2                        Tata Indica Vista Aqua TDI BSIII
3                                Maruti Wagon R LXI Minor
4                                       Chevrolet Beat LT
                              ...                        
4335                              BMW X5 xDrive 30d xLine
4336                        BMW 5 Series 520d Luxury Line
4337                              BMW X5 xDrive 30d xLine
4338             Audi RS7 2015-2019 Sportback Performance
4339    Mercedes-Benz S-Class S 350d Connoisseurs Edition
Name: name, Length: 4340, dtype: object

In [29]:
data['name'].str.split(" ").str[0]

0                Tata
1                Tata
2                Tata
3              Maruti
4           Chevrolet
            ...      
4335              BMW
4336              BMW
4337              BMW
4338             Audi
4339    Mercedes-Benz
Name: name, Length: 4340, dtype: object

In [30]:
data['company_name'] = data['name'].str.split(" ").str[0]
data.head()

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,company_name
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.0,106001,100000,Tata
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.0,108556,100000,Tata
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,Diesel,Individual,Manual,First Owner,11.0,120678,120000,Tata
3,Maruti Wagon R LXI Minor,2010,80000,Petrol,Individual,Manual,Second Owner,9.0,122917,100000,Maruti
4,Chevrolet Beat LT,2010,80000,Petrol,Individual,Manual,Second Owner,10.0,144902,130000,Chevrolet


In [31]:
name = 'Mahendra*Singh*Dhoni'

In [32]:
name.split("*")[2]

'Dhoni'

In [33]:
data['company_name'].nunique()

29

# Numerical Columns -->> Automatic feature engineering

In [34]:
round(data.describe(),2)

,year,km_driven,Rating,ExShowroom Price,selling_price
count,4340.00,4340.00,4340.00,4340.00,4340.00
mean,2013.09,66215.78,11.97,845380.89,504127.31
std,4.22,46644.10,1.91,884840.54,578548.74
min,1992.00,1.00,9.00,106001.00,20000.00
25%,2011.00,35000.00,10.02,445389.50,208749.75
50%,2014.00,60000.00,12.00,596055.00,350000.00
75%,2016.00,90000.00,13.76,946243.00,600000.00
max,2020.00,806599.00,15.00,15538153.00,8900000.00


In [35]:
range_labels = ['new','medium','high','extreme']
range_limits = [0,35000,60000,90000,1000000]

data['km_bins'] = pd.cut(data['km_driven'], labels=range_labels, bins = range_limits)
data.head()

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,company_name,km_bins
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.0,106001,100000,Tata,extreme
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.0,108556,100000,Tata,medium
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,Diesel,Individual,Manual,First Owner,11.0,120678,120000,Tata,high
3,Maruti Wagon R LXI Minor,2010,80000,Petrol,Individual,Manual,Second Owner,9.0,122917,100000,Maruti,high
4,Chevrolet Beat LT,2010,80000,Petrol,Individual,Manual,Second Owner,10.0,144902,130000,Chevrolet,high


In [36]:
# import plotly.express as px

In [37]:
# fig = px.pie(data, names = 'km_bins', values = 'km_driven')
# fig.show()

In [38]:
for col in data.columns:
    if data[col].nunique()/len(data) > 0.05:
        if data[col].dtype == 'object':
            print('Please perform manual feature engineering for -->>', col)
        else:
            print('Auto Feature Engineering for -->>', col)
            new_col_name = col + '_bin'
            data[new_col_name] = pd.qcut(data[col],
                                         4, 
                                         labels = ['b1','b2','b3','b4'])

Please perform manual feature engineering for -->> name
Auto Feature Engineering for -->> km_driven
Auto Feature Engineering for -->> Rating
Auto Feature Engineering for -->> ExShowroom Price
Auto Feature Engineering for -->> selling_price


In [39]:
data

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,company_name,km_bins,km_driven_bin,Rating_bin,ExShowroom Price_bin,selling_price_bin
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.00,106001,100000,Tata,extreme,b4,b1,b1,b1
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.00,108556,100000,Tata,medium,b2,b1,b1,b1
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,Diesel,Individual,Manual,First Owner,11.00,120678,120000,Tata,high,b3,b2,b1,b1
3,Maruti Wagon R LXI Minor,2010,80000,Petrol,Individual,Manual,Second Owner,9.00,122917,100000,Maruti,high,b3,b1,b1,b1
4,Chevrolet Beat LT,2010,80000,Petrol,Individual,Manual,Second Owner,10.00,144902,130000,Chevrolet,high,b3,b1,b1,b1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9407487,4950000,BMW,new,b1,b2,b4,b4
4336,BMW 5 Series 520d Luxury Line,2019,12999,Diesel,Dealer,Automatic,First Owner,9.71,9598350,4800000,BMW,new,b1,b1,b4,b4
4337,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9857238,4950000,BMW,new,b1,b2,b4,b4
4338,Audi RS7 2015-2019 Sportback Performance,2016,13000,Petrol,Dealer,Automatic,First Owner,14.09,14235729,8900000,Audi,new,b1,b4,b4,b4


# Label Encoding

In [40]:
data['fuel'].unique()

array(['Diesel', 'Petrol', 'CNG', 'LPG', 'Electric'], dtype=object)

In [41]:
LN = LabelEncoder()

In [42]:
data['fuel'] = LN.fit_transform(data['fuel'])

In [43]:
data['fuel'].unique()

array([1, 4, 0, 3, 2])

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   name                  4340 non-null   object  
 1   year                  4340 non-null   int64   
 2   km_driven             4340 non-null   int64   
 3   fuel                  4340 non-null   int32   
 4   seller_type           4340 non-null   object  
 5   transmission          4340 non-null   object  
 6   owner                 4340 non-null   object  
 7   Rating                4340 non-null   float64 
 8   ExShowroom Price      4340 non-null   int64   
 9   selling_price         4340 non-null   int64   
 10  company_name          4340 non-null   object  
 11  km_bins               4340 non-null   category
 12  km_driven_bin         4340 non-null   category
 13  Rating_bin            4340 non-null   category
 14  ExShowroom Price_bin  4340 non-null   category
 15  sell

In [45]:
data

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,company_name,km_bins,km_driven_bin,Rating_bin,ExShowroom Price_bin,selling_price_bin
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,1,Individual,Manual,Second Owner,9.00,106001,100000,Tata,extreme,b4,b1,b1,b1
1,Tata Nano Lx BSIV,2012,50000,4,Individual,Manual,Second Owner,9.00,108556,100000,Tata,medium,b2,b1,b1,b1
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,1,Individual,Manual,First Owner,11.00,120678,120000,Tata,high,b3,b2,b1,b1
3,Maruti Wagon R LXI Minor,2010,80000,4,Individual,Manual,Second Owner,9.00,122917,100000,Maruti,high,b3,b1,b1,b1
4,Chevrolet Beat LT,2010,80000,4,Individual,Manual,Second Owner,10.00,144902,130000,Chevrolet,high,b3,b1,b1,b1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,BMW X5 xDrive 30d xLine,2019,30000,1,Dealer,Automatic,First Owner,11.00,9407487,4950000,BMW,new,b1,b2,b4,b4
4336,BMW 5 Series 520d Luxury Line,2019,12999,1,Dealer,Automatic,First Owner,9.71,9598350,4800000,BMW,new,b1,b1,b4,b4
4337,BMW X5 xDrive 30d xLine,2019,30000,1,Dealer,Automatic,First Owner,11.00,9857238,4950000,BMW,new,b1,b2,b4,b4
4338,Audi RS7 2015-2019 Sportback Performance,2016,13000,4,Dealer,Automatic,First Owner,14.09,14235729,8900000,Audi,new,b1,b4,b4,b4


In [46]:
for col in data.columns:
    if ( data[col].dtype == 'object') | hasattr(data[col], 'cat'):
        data[col] = LN.fit_transform(data[col])

In [47]:
data

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,company_name,km_bins,km_driven_bin,Rating_bin,ExShowroom Price_bin,selling_price_bin
0,1228,2010,120000,1,1,1,2,9.00,106001,100000,25,0,3,0,0,0
1,1283,2012,50000,4,1,1,2,9.00,108556,100000,25,2,1,0,0,0
2,1229,2011,70000,1,1,1,0,11.00,120678,120000,25,1,2,1,0,0
3,1041,2010,80000,4,1,1,2,9.00,122917,100000,18,1,2,0,0,0
4,56,2010,80000,4,1,1,2,10.00,144902,130000,3,1,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,39,2019,30000,1,0,0,0,11.00,9407487,4950000,2,3,0,1,3,3
4336,30,2019,12999,1,0,0,0,9.71,9598350,4800000,2,3,0,0,3,3
4337,39,2019,30000,1,0,0,0,11.00,9857238,4950000,2,3,0,1,3,3
4338,25,2016,13000,4,0,0,0,14.09,14235729,8900000,1,3,0,3,3,3


# Correlation

In [48]:
corr = 100*data.corr()
corr.style.background_gradient(cmap = 'coolwarm')

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,company_name,km_bins,km_driven_bin,Rating_bin,ExShowroom Price_bin,selling_price_bin
name,100.000000,-5.190221,12.620297,-8.385037,14.080245,8.778524,3.281915,1.876041,-6.840192,-7.759752,97.451337,-11.630633,11.630633,2.075166,-2.856030,-2.110760
year,-5.190221,100.000000,-41.968815,-12.052816,-9.835151,-14.380043,-41.470517,-0.341515,30.487065,41.392168,-3.972402,48.037822,-48.037822,-0.020174,41.136106,66.112946
km_driven,12.620297,-41.968815,100.000000,-28.563429,11.368942,12.022623,29.711504,-0.392650,-16.510516,-19.228863,13.146088,-84.092423,84.092423,-0.841211,-17.905059,-25.070004
fuel,-8.385037,-12.052816,-28.563429,100.000000,3.838666,4.044484,-1.030093,-1.210894,-23.467424,-26.977883,-10.939373,29.716702,-29.716702,-0.648921,-29.419206,-33.900059
seller_type,14.080245,-9.835151,11.368942,3.838666,100.000000,17.492495,16.568072,-0.527834,-13.274456,-15.155423,14.421752,-12.685058,12.685058,-1.092437,-10.965633,-14.708225
transmission,8.778524,-14.380043,12.022623,4.044484,17.492495,100.000000,7.889262,2.513950,-51.667679,-53.020514,11.069899,-13.422679,13.422679,2.323975,-30.819882,-30.933432
owner,3.281915,-41.470517,29.711504,-1.030093,16.568072,7.889262,100.000000,0.751234,-16.772565,-20.784034,3.645168,-34.241653,34.241653,0.764166,-23.543955,-32.145235
Rating,1.876041,-0.341515,-0.392650,-1.210894,-0.527834,2.513950,0.751234,100.000000,-0.109851,-2.246754,2.165585,0.804309,-0.804309,96.617375,4.854796,-0.098509
ExShowroom Price,-6.840192,30.487065,-16.510516,-23.467424,-13.274456,-51.667679,-16.772565,-0.109851,100.000000,96.062867,-8.755945,19.946381,-19.946381,0.336778,57.968207,54.676162
selling_price,-7.759752,41.392168,-19.228863,-26.977883,-15.155423,-53.020514,-20.784034,-2.246754,96.062867,100.000000,-9.685807,22.965846,-22.965846,-1.622573,57.374834,62.504903


In [49]:
data.drop(columns=['name','Rating', 'ExShowroom Price'], inplace=True)

In [50]:
corr = 100*data.corr()
corr.style.background_gradient(cmap = 'coolwarm')

,year,km_driven,fuel,seller_type,transmission,owner,selling_price,company_name,km_bins,km_driven_bin,Rating_bin,ExShowroom Price_bin,selling_price_bin
year,100.000000,-41.968815,-12.052816,-9.835151,-14.380043,-41.470517,41.392168,-3.972402,48.037822,-48.037822,-0.020174,41.136106,66.112946
km_driven,-41.968815,100.000000,-28.563429,11.368942,12.022623,29.711504,-19.228863,13.146088,-84.092423,84.092423,-0.841211,-17.905059,-25.070004
fuel,-12.052816,-28.563429,100.000000,3.838666,4.044484,-1.030093,-26.977883,-10.939373,29.716702,-29.716702,-0.648921,-29.419206,-33.900059
seller_type,-9.835151,11.368942,3.838666,100.000000,17.492495,16.568072,-15.155423,14.421752,-12.685058,12.685058,-1.092437,-10.965633,-14.708225
transmission,-14.380043,12.022623,4.044484,17.492495,100.000000,7.889262,-53.020514,11.069899,-13.422679,13.422679,2.323975,-30.819882,-30.933432
owner,-41.470517,29.711504,-1.030093,16.568072,7.889262,100.000000,-20.784034,3.645168,-34.241653,34.241653,0.764166,-23.543955,-32.145235
selling_price,41.392168,-19.228863,-26.977883,-15.155423,-53.020514,-20.784034,100.000000,-9.685807,22.965846,-22.965846,-1.622573,57.374834,62.504903
company_name,-3.972402,13.146088,-10.939373,14.421752,11.069899,3.645168,-9.685807,100.000000,-11.946879,11.946879,2.320414,-3.121876,-2.511600
km_bins,48.037822,-84.092423,29.716702,-12.685058,-13.422679,-34.241653,22.965846,-11.946879,100.000000,-100.000000,1.114484,23.448379,30.692995
km_driven_bin,-48.037822,84.092423,-29.716702,12.685058,13.422679,34.241653,-22.965846,11.946879,-100.000000,100.000000,-1.114484,-23.448379,-30.692995


# Train Test Split

In [51]:
x = data.drop(columns = 'selling_price')
y = data['selling_price']

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state = 1234)

In [53]:
len(x_train)

3038

In [54]:
len(y_train)

3038

# Model Development

    1. Model Name
    2. Training
    3. Exam - Predicted Answers
    4. Accuracy Check

##### Step 1. Model Declaration

In [55]:
model = LinearRegression()

##### Step 2. Model training (fitting)

In [56]:
model.fit(x_train,y_train)

LinearRegression()

##### Step 3. Prediction

In [57]:
y_pred = model.predict(x_test)

In [58]:
y_pred

array([1438411.5968826 ,  287860.18461718,  294504.20672931, ...,
       1405617.85825227,  787988.93414171,  652938.87056424])

##### Step 4. Accuracy

In [59]:
accuracy = 100* r2_score(y_test, y_pred)
accuracy

56.337831680288886

# Running all model

In [60]:
models = [LinearRegression(), Lasso(), ElasticNet(), KNeighborsRegressor(),
         DecisionTreeRegressor(), RandomForestRegressor(), AdaBoostRegressor(),
          GradientBoostingRegressor()]

In [61]:
acc_dic = {} 

In [62]:
for model in models:
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    accuracy = 100* r2_score(y_test, y_pred)
    acc_dic[model] = round(accuracy,2)
    

In [63]:
acc_dic

{LinearRegression(): 56.34,
 Lasso(): 56.34,
 ElasticNet(): 46.28,
 KNeighborsRegressor(): 55.13,
 DecisionTreeRegressor(): 53.34,
 RandomForestRegressor(): 82.97,
 AdaBoostRegressor(): 56.2,
 GradientBoostingRegressor(): 82.1}